<a href="https://colab.research.google.com/github/Vedant-Sahu/TrackML/blob/master/dataset_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install git+git://github.com/sibirrer/lenstronomy.git
!pip install lenstronomy==0.9.2

     |████████████████████████████████| 5.8MB 3.1MB/s 
  Created wheel for lenstronomy: filename=lenstronomy-0.9.2-cp36-none-any.whl size=5960558 sha256=125531d4b3f8c60a095a71eee634c0acc692b2ad798dd3f22c857c12430be958
  Stored in directory: /root/.cache/pip/wheels/4a/96/56/e76a29508e3d79f3ecac06fcebc3915643e911481eb3c46d20
Successfully built lenstronomy


In [2]:
!pip install git+git://github.com/sibirrer/fastell4py.git

  Cloning git://github.com/sibirrer/fastell4py.git to /tmp/pip-req-build-7bauay0b
  Running command git clone -q git://github.com/sibirrer/fastell4py.git /tmp/pip-req-build-7bauay0b
  Created wheel for fastell4py: filename=fastell4py-0.1.0-cp36-cp36m-linux_x86_64.whl size=199464 sha256=02a203ff5741c56ba5e42ac5ba35c8167d00f13280e94ee61ee777844ff9d4be
  Stored in directory: /tmp/pip-ephem-wheel-cache-6pwsex2c/wheels/1b/47/5f/d49140839c4ab156bdfd22f629de748c5b0b18f5c1ebc3caab
Successfully built fastell4py


In [3]:
!pip install aplpy

     |████████████████████████████████| 92kB 3.6MB/s 
     |████████████████████████████████| 389kB 10.9MB/s 
     |████████████████████████████████| 1.1MB 61.8MB/s 
     |████████████████████████████████| 1.3MB 40.8MB/s 
     |████████████████████████████████| 204kB 42.3MB/s 
  Created wheel for pyregion: filename=pyregion-2.0-cp36-cp36m-linux_x86_64.whl size=308447 sha256=cbc7d574f643e18b23612cd4eedd9ac60c4d470a17b121c1c98bff933c297e02
  Stored in directory: /root/.cache/pip/wheels/ca/2c/2b/c95350df7d42ba9044b7528c5ba549bead550433b189008a7f
Successfully built pyregion


In [4]:
!pip install git+git://github.com/waspinator/pycococreator.git@0.2.0

  Cloning git://github.com/waspinator/pycococreator.git (to revision 0.2.0) to /tmp/pip-req-build-btlcdmzj
  Running command git clone -q git://github.com/waspinator/pycococreator.git /tmp/pip-req-build-btlcdmzj
  Running command git checkout -q fba8f4098f3c7aaa05fe119dc93bbe4063afdab8
  Created wheel for pycococreatortools: filename=pycococreatortools-0.2.0-cp36-none-any.whl size=7192 sha256=c78433da580a0d50b6afc91124375a11e39165608ef133cb4da8a95ed1960bd7
  Stored in directory: /tmp/pip-ephem-wheel-cache-c88xdbmb/wheels/ad/8a/49/9cda5b77c0aec14230df33a9ddfaf60bf488f08a311a2aec0f
Successfully built pycococreatortools


In [0]:
import copy
import numpy as np
import aplpy
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

# lenstronomy module import
import lenstronomy.Util.data_util as data_util
import lenstronomy.Util.util as util
import lenstronomy.Plots.plot_util as plot_util
import lenstronomy.Util.param_util as param_util
from lenstronomy.SimulationAPI.sim_api import SimAPI
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver

import datetime
import json
import os
import re
import fnmatch
import pycocotools
from pycococreatortools import pycococreatortools
from pycocotools.coco import COCO
import skimage.io as io
import pylab

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def filter_for_npy(root, files):
    
    file_types = ['*.npy']
    file_types = r'|'.join([fnmatch.translate(x) for x in file_types])
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    
    return files

def filter_for_annotations(root, files, image_filename):
    
    file_types = ['*.png']
    file_types = r'|'.join([fnmatch.translate(x) for x in file_types])
    basename_no_extension = os.path.splitext(os.path.basename(image_filename))[0]
    file_name_prefix = basename_no_extension + '.*'
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    files = [f for f in files if re.match(file_name_prefix, os.path.splitext(os.path.basename(f))[0])]

    return files

def dataset_creation(n, train_val):
  
    ROOT_DIR = '/content/gdrive/My Drive/mrcnn_data_01.22.20'
    
    if train_val == "train":
        IMAGE_DIR = os.path.join(ROOT_DIR, "images_train")
        ANNOTATION_DIR = os.path.join(ROOT_DIR, "annotations_train")
        json_file = os.path.join(ROOT_DIR, "instances_mrcnn_train.json" )
    elif train_val == "val":
        IMAGE_DIR = os.path.join(ROOT_DIR, "images_val")
        ANNOTATION_DIR = os.path.join(ROOT_DIR, "annotations_val")
        json_file = os.path.join(ROOT_DIR, "instances_mrcnn_val.json" )
    
    file = open("/content/gdrive/My Drive/CNN_Data/lens_outputs_new.pickle", "rb")
    lens_outputs = pickle.load(file, encoding='latin1')
    
    sersic_list = np.random.choice([1, 2], n, p = [0.5, 0.5])
    satellite_list = np.random.choice([0, 1], n, p = [0.67, 0.33])
    
    # Instrument setting
    HST_camera = {'pixel_scale': 0.05, # scale (in arcseonds) of pixels
                  }
    
    numpix = 128  # number of pixels per axis of the image to be modelled
    
    # Settings for the different bands
    read_noise = [14, 3, 3]
    ccd_gain = [2.5, 1.5, 1.5]
    exposure_time = [2196.9, 1429, 1119.2]
    sky_brightness = [22.9617, 23.1502, 24.1893]
    magnitude_zero_point  = [24.9514, 24.6997, 25.7388]
    num_exposures = [1, 1, 1]
    seeing = [0, 0, 0]
    psf_lens_index = np.random.randint(0, 12, n)
    
    # Randomized settings for the lens
    phi_spemd = np.random.uniform(0, np.pi, n)
    q_spemd = np.random.uniform(0.65, 0.98, n)
    theta_E = np.random.uniform(0.75, 1.5, n)
    gamma = np.random.uniform(1.9, 2.1, n)
    phi_shear = np.random.uniform(0, np.pi, n)
    gamma_shear = np.random.uniform(0, 0.15, n)
    x_lens = np.random.uniform(-0.5, 0.5, n)
    y_lens = np.random.uniform(-0.5, 0.5, n)
    
    # Randomized settings for the source
    e1_source = np.random.uniform(-0.4, 0.4, n)
    e2_source = np.random.uniform(-0.4, 0.4, n)
    R_source = np.random.uniform(0.08, 0.25, n)
    n_source = np.random.uniform(0.8, 2.5, n)
    mag_source = np.random.uniform(18, 26, n)
    mag_ps = np.random.uniform(19.5, 23, n)
    ra_source = x_lens + np.random.normal(0, 0.1, n)
    dec_source = y_lens + np.random.normal(0, 0.1, n)
    
    # Randomized settings for the difference between the three bands       
    F160W_F814W_source = np.random.uniform(-4.5, 1.5, n)  # color mag_F160W - mag_F814W for source
    F160W_F814W_lens = np.random.uniform(-2.5, -0.75, n)  # color mag_F160W - mag_F814W for lens light
    F160W_F814W_ps = np.random.uniform(-1.5, 0.1, n)
    F160W_F475X_source = np.random.uniform(-5, 1, n)
    F160W_F475X_lens = F160W_F814W_lens + np.random.uniform(-4, -1.5, n)
    F160W_F475X_ps = np.random.uniform(-2.5, 0.1, n)
    
    
        
    for i in range(0,n):
      

        HST_F160W_band_obs = {'read_noise': read_noise[0], # std of noise generated by read-out (in units of electrons)
                           'ccd_gain': ccd_gain[0], # electrons/ADU (analog-to-digital unit). A gain of 8 means that the camera digitizes the CCD signal so that each ADU corresponds to 8 photoelectrons.
                           'exposure_time': exposure_time[0],  # exposure time per image (in seconds)
                           'sky_brightness': sky_brightness[0],  # sky brightness (in magnitude per square arcseconds)
                           'magnitude_zero_point': magnitude_zero_point[0],  # magnitude in which 1 count per second per arcsecond square is registered (in ADU's)
                           'num_exposures': num_exposures[0],  # number of exposures that are combined
                           'seeing': seeing[0],  # full width at half maximum of the PSF (if not specific psf_model is specified)
                           'psf_type': 'PIXEL',  # string, type of PSF ('GAUSSIAN' and 'PIXEL' supported)
                           'psf_model': lens_outputs[psf_lens_index[i]]['multi_band_list'][0][1]['kernel_point_source']  # 2d numpy array, model of PSF centered with odd number of pixels per axis (optional when psf_type='PIXEL' is chosen)
                          }

        HST_F814W_band_obs = {'read_noise': read_noise[1],
                           'ccd_gain': ccd_gain[1],
                           'exposure_time': exposure_time[1],
                           'sky_brightness': sky_brightness[1],
                           'magnitude_zero_point': magnitude_zero_point[1],
                           'num_exposures': num_exposures[1],
                           'seeing': seeing[1],
                           'psf_type': 'PIXEL',
                           'psf_model': lens_outputs[psf_lens_index[i]]['multi_band_list'][1][1]['kernel_point_source']
                           }

        HST_F475X_band_obs = {'read_noise': read_noise[2], 
                           'ccd_gain': ccd_gain[2],                   
                           'exposure_time': exposure_time[2],
                           'sky_brightness': sky_brightness[2],
                           'magnitude_zero_point': magnitude_zero_point[2],
                           'num_exposures': num_exposures[2],
                           'seeing': seeing[2],
                           'psf_type': 'PIXEL',
                           'psf_model': lens_outputs[psf_lens_index[i]]['multi_band_list'][1][1]['kernel_point_source']   
                          }

        # lenstronomy provides these setting to be imported with the SimulationAPI.observation_constructor routines.

        sersic = sersic_list[i]
        satellite_present = satellite_list[i]

        e1_spemd, e2_spemd = param_util.phi_q2_ellipticity(phi=phi_spemd[i], q=q_spemd[i])
        e1_shear, e2_shear = param_util.phi_gamma_ellipticity(phi=phi_shear[i], gamma=gamma_shear[i]) 

        lens_model_list = ['SPEMD', 'SHEAR']
        kwargs_spemd = {'theta_E': theta_E[i], 'e1': e1_spemd, 'e2': e2_spemd, 'center_x': x_lens[i], 'center_y': y_lens[i], 'gamma': gamma[i]}  # SPEMD model
        kwargs_shear = {'e1': e1_shear, 'e2': e2_shear}  # SHEAR model
        kwargs_lens = [kwargs_spemd, kwargs_shear]

        if sersic == 1:

            phi = np.random.uniform(0,np.pi)
            q = np.random.uniform(0.6,1)
            e1, e2 = param_util.phi_q2_ellipticity(phi=phi, q=q)
            n_1 = np.random.uniform(0.6,6)
            R_1 = np.random.uniform(0.4,2)
            mag = np.random.uniform(16,20.5)
         
            lens_light_model_list = ['SERSIC_ELLIPSE']
            kwargs_lens_light_mag_F160W = [{'magnitude': mag, 'R_sersic': R_1, 'n_sersic': n_1, 'e1': e1, 'e2': e2, 'center_x': x_lens[i], 'center_y': y_lens[i]}]
            lens_ann_filename = "%05d"% i + "_single_sersic_0.png"
            
        elif sersic == 2:

            phi_1 = np.random.uniform(0,np.pi)
            q_1 = np.random.uniform(0.6,1)
            e1_1, e2_1 = param_util.phi_q2_ellipticity(phi=phi_1, q=q_1)
            n_1 = np.random.uniform(0.6,2)
            R_1 = np.random.uniform(1,2)
            mag_1 = np.random.uniform(16,20.5)

            phi_2 = np.random.uniform(0,np.pi)
            q_2 = np.random.uniform(0.6,1)
            e1_2, e2_2 = param_util.phi_q2_ellipticity(phi=phi_2, q=q_2)
            n_2 = np.random.uniform(1.5,6)
            R_2 = np.random.uniform(0.4,1.5)
            mag_2 = np.random.uniform(18,20)

            lens_light_model_list = ['SERSIC_ELLIPSE', 'SERSIC_ELLIPSE']
            kwargs_lens_light_mag_F160W = [{'magnitude': mag_1, 'R_sersic': R_1, 'n_sersic': n_1, 'e1': e1_1, 'e2': e2_1, 'center_x': x_lens[i], 'center_y': y_lens[i]},
                                       {'magnitude': mag_2, 'R_sersic': R_2, 'n_sersic': n_2, 'e1': e1_2, 'e2': e2_2, 'center_x': x_lens[i], 'center_y': y_lens[i]}]        
            lens_ann_filename = "%05d"% i + "_double_sersic_0.png"
            
        lens_mask = np.zeros(shape=[128, 128])
        x = int(x_lens[i]*20+58)
        y = int(y_lens[i]*20+58)
        lens_mask[x:x+12,y:y+12] = 255
        lens_ann = Image.fromarray(lens_mask)
        lens_ann = lens_ann.convert("L")
        lens_ann_file = os.path.join(ANNOTATION_DIR, lens_ann_filename)
        lens_ann.save(lens_ann_file)
        
        
        if satellite_present == 1:

            lens_model_list = ['SPEMD', 'SHEAR', 'NIE']

            theta_E_nie = theta_E[i] * np.random.uniform(0.1,0.4)
            e1_sat = np.random.uniform(-0.1,0.1)
            e2_sat = np.random.uniform(-0.1,0.1)
            x_sat = x_lens[i]+np.random.uniform(-1,1)
            y_sat = y_lens[i]+np.random.uniform(-1,1)
            R_sat = np.random.uniform(0.1,1)
            n_sat = np.random.uniform(3,9)
            mag_sat = np.random.uniform(20,24)
            
            sat_ann_filename = "%05d"% i + "_satellite.png"
            sat_mask = np.zeros(shape=[128, 128])
            x = int(x_sat*20+60)
            y = int(y_sat*20+60)
            sat_mask[x:x+8,y:y+8] = 255
            sat_ann = Image.fromarray(sat_mask)
            sat_ann = sat_ann.convert("L")
            
            sat_ann_file = os.path.join(ANNOTATION_DIR, sat_ann_filename)
            sat_ann.save(sat_ann_file) 

            kwargs_nie = {'theta_E': theta_E_nie, 'e1': e1_sat, 'e2': e2_sat, 's_scale': 0.0000000001}  #NIE model 
            kwargs_lens.append(kwargs_nie)

            lens_light_model_list.append('SERSIC_ELLIPSE')
            kwargs_sat_light = {'magnitude': mag_sat, 'R_sersic': R_sat, 'n_sersic': n_sat, 'e1': e1_sat, 'e2': e2_sat, 'center_x': x_sat, 'center_y': y_sat}
            kwargs_lens_light_mag_F160W.append(kwargs_sat_light)


        kwargs_model = {'lens_model_list': lens_model_list,  # list of lens models to be used
                        'lens_light_model_list': lens_light_model_list,  # list of unlensed light models to be used
                        'source_light_model_list': ['SERSIC_ELLIPSE'],  # list of extended source models to be used
                        'point_source_model_list': ['SOURCE_POSITION']  # list of point source models to be used
                       }

        # here we merge observations and instrument for the three bands
        kwargs_F160W_band = util.merge_dicts(HST_camera, HST_F160W_band_obs)
        kwargs_F814W_band = util.merge_dicts(HST_camera, HST_F814W_band_obs)
        kwargs_F475X_band = util.merge_dicts(HST_camera, HST_F475X_band_obs)

        # here we define the numerical options used in the ImSim module. 
        # Have a look at the ImageNumerics class for detailed descriptions.
        # If not further specified, the default settings are used.
        kwargs_numerics = {'point_source_supersampling_factor': 1}

        sim_F160W = SimAPI(numpix=numpix, kwargs_single_band=kwargs_F160W_band, kwargs_model=kwargs_model, kwargs_numerics=kwargs_numerics)
        sim_F814W = SimAPI(numpix=numpix, kwargs_single_band=kwargs_F814W_band, kwargs_model=kwargs_model, kwargs_numerics=kwargs_numerics)
        sim_F475X = SimAPI(numpix=numpix, kwargs_single_band=kwargs_F475X_band, kwargs_model=kwargs_model, kwargs_numerics=kwargs_numerics)

        # return the ImSim instance. With this class instance, you can compute all the
        # modelling accessible of the core modules. See class documentation and other notebooks.
        imSim_F160W = sim_F160W.image_model_class
        imSim_F814W = sim_F814W.image_model_class
        imSim_F475X = sim_F475X.image_model_class

        lens_model_class = LensModel(lens_model_list=lens_model_list)
        lensEquationSolver = LensEquationSolver(lens_model_class)
        x_image, y_image = lensEquationSolver.findBrightImage(ra_source[i], dec_source[i], kwargs_lens, numImages = 4,
                                                             min_distance = HST_camera['pixel_scale'], search_window = numpix * HST_camera['pixel_scale'])

        quasar_mask = np.zeros(shape=[128, 128, len(x_image)])
        
        for j in range(len(x_image)):
            x = int(x_image[j]*20+60)
            y = int(y_image[j]*20+60)
            quasar_mask[x:x+8,y:y+8,j] = 255
            ann = Image.fromarray(quasar_mask[:,:,j])
            ann = ann.convert("L")
            ann_filename = "%05d"% i + "_quasar_image_" + str(j) + ".png"
            ann_file = os.path.join(ANNOTATION_DIR, ann_filename)
            ann.save(ann_file)
            
        # F160W-band

        # source light
        kwargs_source_mag_F160W = [{'magnitude': mag_source[i], 'R_sersic': R_source[i], 'n_sersic': n_source[i], 'e1': e1_source[i], 'e2': e2_source[i],
                                'center_x': ra_source[i], 'center_y': dec_source[i]}]
        # point source
        kwargs_ps_mag_F160W = [{'magnitude': mag_ps[i], 'ra_source': ra_source[i], 'dec_source': dec_source[i]}]


        # and now we define the colors of the other two bands

        # F814W-band
        
        kwargs_lens_light_mag_F814W = copy.deepcopy(kwargs_lens_light_mag_F160W)
        kwargs_lens_light_mag_F814W[0]['magnitude'] -= F160W_F814W_lens[i]

        kwargs_source_mag_F814W = copy.deepcopy(kwargs_source_mag_F160W)
        kwargs_source_mag_F814W[0]['magnitude'] -= F160W_F814W_source[i]

        kwargs_ps_mag_F814W = copy.deepcopy(kwargs_ps_mag_F160W)
        kwargs_ps_mag_F814W[0]['magnitude'] -= F160W_F814W_source[i]

        # F475X-band
        
        kwargs_lens_light_mag_F475X = copy.deepcopy(kwargs_lens_light_mag_F160W)
        kwargs_lens_light_mag_F475X[0]['magnitude'] -= F160W_F475X_lens[i]

        kwargs_source_mag_F475X = copy.deepcopy(kwargs_source_mag_F160W)
        kwargs_source_mag_F475X[0]['magnitude'] -= F160W_F475X_source[i]

        kwargs_ps_mag_F475X = copy.deepcopy(kwargs_ps_mag_F160W)
        kwargs_ps_mag_F475X[0]['magnitude'] -= F160W_F475X_source[i]

        if sersic == 2:
            F160W_F814W_double = np.random.uniform(-2,-0.5) 
            F160W_F475X_double = F160W_F814W_double + np.random.uniform(-4,-1)
            kwargs_lens_light_mag_F814W[1]['magnitude'] -= F160W_F814W_double
            kwargs_lens_light_mag_F475X[1]['magnitude'] -= F160W_F475X_double

        if satellite_present == 1:    
            F160W_F814W_sat = np.random.uniform(-2.5,-0.1) 
            F160W_F475X_sat = F160W_F814W_sat + np.random.uniform(-5,-2)
            kwargs_lens_light_mag_F814W[-1]['magnitude'] -= F160W_F814W_sat
            kwargs_lens_light_mag_F475X[-1]['magnitude'] -= F160W_F475X_sat

        # turn magnitude kwargs into lenstronomy kwargs
        kwargs_lens_light_F160W, kwargs_source_F160W, kwargs_ps_F160W = sim_F160W.magnitude2amplitude(kwargs_lens_light_mag_F160W, kwargs_source_mag_F160W, kwargs_ps_mag_F160W)
        kwargs_lens_light_F814W, kwargs_source_F814W, kwargs_ps_F814W = sim_F814W.magnitude2amplitude(kwargs_lens_light_mag_F814W, kwargs_source_mag_F814W, kwargs_ps_mag_F814W)
        kwargs_lens_light_F475X, kwargs_source_F475X, kwargs_ps_F475X = sim_F475X.magnitude2amplitude(kwargs_lens_light_mag_F475X, kwargs_source_mag_F475X, kwargs_ps_mag_F475X)
        
        image_F160W = imSim_F160W.image(kwargs_lens, kwargs_source_F160W, kwargs_lens_light_F160W, kwargs_ps_F160W)
        image_F814W = imSim_F814W.image(kwargs_lens, kwargs_source_F814W, kwargs_lens_light_F814W, kwargs_ps_F814W)
        image_F475X = imSim_F475X.image(kwargs_lens, kwargs_source_F475X, kwargs_lens_light_F475X, kwargs_ps_F475X)

        # add noise
        image_F160W += sim_F160W.noise_for_model(model=image_F160W)
        image_F814W += sim_F814W.noise_for_model(model=image_F814W)
        image_F475X += sim_F475X.noise_for_model(model=image_F475X)

        # and plot it

        img = np.zeros((image_F160W.shape[0], image_F160W.shape[1], 3), dtype=float)

        img[:,:,0] = np.transpose(image_F160W)
        img[:,:,1] = np.transpose(image_F814W)
        img[:,:,2] = np.transpose(image_F475X)
        img = np.arcsinh(np.arcsinh(np.arcsinh(img)))
        file = os.path.join(IMAGE_DIR, "%05d"% i)
        np.save(file, img)
        
        if (i+1)%100 == 0:
            print("Creating images - "+str(i+1)+"/"+str(n))

    INFO = {
        "description": "Gravitational Lensing Dataset",
        "url": "https://github.com/Vedant-Sahu",
        "version": "01.22.20",
        "year": 2020,
        "contributor": "Vedant-Sahu",
        "date_created": datetime.datetime.utcnow().isoformat(' ')
    }

    LICENSES = [
        {
            "id": 1,
            "name": " ",
            "url": " "
        }
    ]

    CATEGORIES = [
        {
            'id': 1,
            'name': 'single_sersic',
        },
        {
            'id': 2,
            'name': 'double_sersic',
        },
        {
            'id': 3,
            'name': 'quasar_image',
        },
        {
            'id': 4,
            'name': 'satellite',
        }
    ]

    coco_output = {
        "info": INFO,
        "licenses": LICENSES,
        "categories": CATEGORIES,
        "images": [],
        "annotations": []
    }

    image_id = 1
    segmentation_id = 1
    
    # filter for npy images
    for root, _, files in os.walk(IMAGE_DIR):
        image_files = filter_for_npy(root, files)

        # go through each image
        for image_filename in image_files:
            image = np.load(image_filename)
            image_size = (image.shape[0], image.shape[1])
            image_info = pycococreatortools.create_image_info(image_id, os.path.basename(image_filename),
                            image_size, coco_url=image_filename)
            coco_output["images"].append(image_info)

            # filter for associated png annotations
            for root, _, files in os.walk(ANNOTATION_DIR):
                annotation_files = filter_for_annotations(root, files, image_filename)

                # go through each associated annotation
                for annotation_filename in annotation_files:
                    
                    class_id = [x['id'] for x in CATEGORIES if x['name'] in annotation_filename][0]

                    category_info = {'id': class_id, 'is_crowd': 'crowd' in image_filename}
                    binary_mask = np.asarray(Image.open(annotation_filename)).astype(np.uint8)
                    
                    annotation_info = pycococreatortools.create_annotation_info(
                        segmentation_id, image_id, category_info, binary_mask,
                        image_size, tolerance=2)
                                     
                    if annotation_info is not None:
                        coco_output["annotations"].append(annotation_info)

                    segmentation_id = segmentation_id + 1

            if image_id%100 == 0:
                print("Creating annotations - "+str(image_id)+"/"+str(n))
            
            image_id = image_id + 1

    with open(json_file, 'w') as output_json_file:
        json.dump(coco_output, output_json_file)
